# Predicting Chicago Taxi Pickup Density using Random Forest

## Importing libraries

In [1]:
import os
os.chdir('/Users/Kevin/Documents/TAMIDS Taxi Data')

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import random
import math
import datetime
import time

## Data

In [2]:
x_train = pd.read_csv('PickupData.csv')
x_test = pd.read_csv('PickupDataTest.csv')

A preview of our data.

In [3]:
x_train.head()

,Year_num,Month,Hour,Day,WDay,Latitude,Longitude,Pickups,Month_num,Hour_num,Day_num,Year_sin,Year_cos,Month_sin,Month_cos,Hour_sin,Hour_cos,Day_sin,Day_cos
0,0.202740,3,17,15,5,41.879255,-87.642649,2,0.466667,0.750000,0.678571,0.956235,0.292600,0.207912,-0.978148,-1.000000,-1.836970e-16,-0.900969,-0.433884
1,0.106849,2,19,8,5,41.893216,-87.637844,1,0.233333,0.833333,0.690476,0.622047,0.782980,0.994522,0.104528,-0.866025,5.000000e-01,-0.930874,-0.365341
2,0.339726,5,20,4,6,41.916005,-87.675095,3,0.100000,0.875000,0.839286,0.845249,-0.534373,0.587785,0.809017,-0.707107,7.071068e-01,-0.846724,0.532032
3,0.695890,9,12,11,3,41.884987,-87.620993,4,0.333333,0.541667,0.363095,-0.942761,-0.333469,0.866025,-0.500000,-0.258819,-9.659258e-01,0.757972,-0.652287
4,0.953425,12,21,14,6,41.906026,-87.675312,2,0.433333,0.916667,0.845238,-0.288482,0.957485,0.406737,-0.913545,-0.500000,8.660254e-01,-0.826239,0.563320


### Defining Time Variables
The time granularity with respect to the granularity of location will be defined as bins per day (hours per day).  We must adjust our timestamps accordingly. Note that our data was processed in R.
* **Hour_num**: Using a 24-hour clock, the hours will be defined from 1-24 (1:00am is 1, 12:00am is 24). Using the **strptime()** function, the hours are defined from 0-23, so we just add 1 to correct this. The formula for this calculation is:  
\begin{equation}
HourNum = (Hour+1)/24
\end{equation}

* **Day_num**: The day of the week will be defined from 0-6 (Monday is 0, Sunday is 6), and we will incorporate hour of day in this calculation also. The formula for this calculation is:
\begin{equation}
DayNum = (((WDay-1) \mod 7)+(Hour+1)/24)/7
\end{equation}
**WDay** is the day of the week.
Example: Friday, 2:00pm (Day = 5, Hour = 14)
\begin{equation}
    DayNum = ((5 \mod 7)+(14+1)/24)/7
\end{equation}
We do **WDay**-1 to count all of the days of the week previous to Friday. Since our range of integers for week is defined on the interval [0,6] we will be doing calculations for day of the week on mod 7. Then finally we divide by 7, the number of days in a week.

* **Month_num**: The day of the month.
\begin{equation}
MonthNum = (Day-1)/30
\end{equation}

* **Year_num**: The day of the year will defined on a scale of 1-365. The **strptime()** function defines the day of the year from 0-364, so like hour, we add an correction factor of 1. The formula for this calculation is:
\begin{equation}
YearNum = (YearDay+1)/365
\end{equation}

Note: For 2016, we used 366 to count since it is a leap year.

* **Sine** and **Cosine**: We capture cyclicality by applying sine and cosine functions to our calculated time, month, day, and year numbers.

### Create response variables

In [4]:
y_pickup_train = np.log10(x_train['Pickups']+1)
y_pickup_test = np.log10(x_test['Pickups']+1)

In [5]:
# Remove pickup and wday column from training and testing data
pickup_train = x_train.drop(['Pickups', 'WDay'], axis=1)
pickup_test = x_test.drop(['Pickups', 'WDay'], axis=1)

## Model Building
### Optimizing the random forest hyper-parameters
Before we run our model, we want to find the best parameters to run the model with, in particular, we want to optimize the number of trees to use, the depth of the trees, and the number of variables considered at each split. We will use a 5-fold cross validation function to find the best parameters. 

In [6]:
def optimize_params(clf, params, X, y):
    cv = GridSearchCV(clf, param_grid=params, n_jobs=1, cv=5, verbose=3)
    cv.fit(X, y)
    print('Best parameters:', cv.best_params_)
    print('Best score:', cv.best_score_)
    print('Best estimators:', cv.best_estimator_)

#### Defining the range of parameters
Note that **n_estimators** is the number of trees, **max_features** is the number of variables considered at each split, and **max_depth** is the depth of the trees.

In [91]:
rf_estimator = RandomForestRegressor(n_estimators=20, n_jobs=-1)

params = {'n_estimators': [1, 2, 5, 10, 30, 50, 100],
          'max_features': ['auto', 'log2'],
          'max_depth': [10, 20, 50, 100]
         }

start_time = time.time()
best_params = optimize_params(rf_estimator, params, pickup_train, y_pickup_train)
time.time() - start_time

Fitting 5 folds for each of 56 candidates, totalling 280 fits
[CV] max_depth=10, max_features=auto, n_estimators=1 .................
[CV]  max_depth=10, max_features=auto, n_estimators=1, score=0.559440, total=   2.2s
[CV] max_depth=10, max_features=auto, n_estimators=1 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV]  max_depth=10, max_features=auto, n_estimators=1, score=0.612251, total=   2.2s
[CV] max_depth=10, max_features=auto, n_estimators=1 .................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.6s remaining:    0.0s


[CV]  max_depth=10, max_features=auto, n_estimators=1, score=0.615317, total=   2.1s
[CV] max_depth=10, max_features=auto, n_estimators=1 .................
[CV]  max_depth=10, max_features=auto, n_estimators=1, score=0.611700, total=   2.1s
[CV] max_depth=10, max_features=auto, n_estimators=1 .................
[CV]  max_depth=10, max_features=auto, n_estimators=1, score=0.558787, total=   2.1s
[CV] max_depth=10, max_features=auto, n_estimators=2 .................
[CV]  max_depth=10, max_features=auto, n_estimators=2, score=0.565525, total=   2.6s
[CV] max_depth=10, max_features=auto, n_estimators=2 .................
[CV]  max_depth=10, max_features=auto, n_estimators=2, score=0.616767, total=   2.6s
[CV] max_depth=10, max_features=auto, n_estimators=2 .................
[CV]  max_depth=10, max_features=auto, n_estimators=2, score=0.620318, total=   2.6s
[CV] max_depth=10, max_features=auto, n_estimators=2 .................
[CV]  max_depth=10, max_features=auto, n_estimators=2, score=0.6

...

[CV]  max_depth=100, max_features=auto, n_estimators=1, score=0.196569, total=   4.0s
[CV] max_depth=100, max_features=auto, n_estimators=2 ................
[CV]  max_depth=100, max_features=auto, n_estimators=2, score=0.500745, total=   3.7s
[CV] max_depth=100, max_features=auto, n_estimators=2 ................
[CV]  max_depth=100, max_features=auto, n_estimators=2, score=0.744274, total=   3.7s
[CV] max_depth=100, max_features=auto, n_estimators=2 ................
[CV]  max_depth=100, max_features=auto, n_estimators=2, score=0.758840, total=   4.0s
[CV] max_depth=100, max_features=auto, n_estimators=2 ................
[CV]  max_depth=100, max_features=auto, n_estimators=2, score=0.593082, total=   4.0s
[CV] max_depth=100, max_features=auto, n_estimators=2 ................
[CV]  max_depth=100, max_features=auto, n_estimators=2, score=0.359406, total=   3.7s
[CV] max_depth=100, max_features=auto, n_estimators=5 ................
[CV]  max_depth=100, max_features=auto, n_estimators=5, sc

[CV]  max_depth=100, max_features=log2, n_estimators=30, score=0.839244, total=   9.6s
[CV] max_depth=100, max_features=log2, n_estimators=30 ...............
[CV]  max_depth=100, max_features=log2, n_estimators=30, score=0.853031, total=  10.0s
[CV] max_depth=100, max_features=log2, n_estimators=30 ...............
[CV]  max_depth=100, max_features=log2, n_estimators=30, score=0.719004, total=  12.4s
[CV] max_depth=100, max_features=log2, n_estimators=30 ...............
[CV]  max_depth=100, max_features=log2, n_estimators=30, score=0.507412, total=  10.4s
[CV] max_depth=100, max_features=log2, n_estimators=50 ...............
[CV]  max_depth=100, max_features=log2, n_estimators=50, score=0.651389, total=  14.3s
[CV] max_depth=100, max_features=log2, n_estimators=50 ...............
[CV]  max_depth=100, max_features=log2, n_estimators=50, score=0.842260, total=  16.1s
[CV] max_depth=100, max_features=log2, n_estimators=50 ...............
[CV]  max_depth=100, max_features=log2, n_estimators

[Parallel(n_jobs=1)]: Done 280 out of 280 | elapsed: 558.9min finished


Best parameters: {'max_depth': 50, 'max_features': 'log2', 'n_estimators': 100}
Best score: 0.719075403196
Best estimators: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


33574.71210980415

After performing cross validation, here are the results we found:
* **max_depth** = 50
* **max_features** = log2
* **n_estimators** = 100

### Fitting the model
We will use the parameters we found in the previous to fit our random forest model.

In [6]:
reg = RandomForestRegressor(n_estimators=100, max_depth=50, max_features='log2', n_jobs=1)

In [7]:
reg.fit(pickup_train, y_pickup_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=50,
           max_features='log2', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

### Make predictions and evaluating results

In [8]:
predicted_pickups = reg.predict(pickup_test)
accuracy_score(np.round(np.power(10, y_pickup_test)-1, decimals=0),
               np.round(np.power(10, predicted_pickups)-1, decimals=0))

0.43662059133453168

Performing prediction on the 2017 test data, we get only a 43.66% accuracy rate, not so good. However, for our particular study, a low prediction accuracy does not necessary mean that our model is not good. This accuracy score only accounts for the number of pickups that our model has predicted exactly and does not factor in the proximity of our prediction, and considering the scale of the data we are working with, the proximity to the exact solution is pretty important to asses. For example, if a specific location had 1,000 true pickups, but our model predicted that same location would have 998 pickups. That prediction is not exactly correct but since we are working with a dataset containing several million pickups, a prediction that is off by only a factor of 2 is fairly accurate. So to take a further inspection on how good our model is, we must take into account how far are predictions are off from the true values and we can do that by calculating the root mean square error.

#### Calculate RMSE

In [10]:
rmse = np.sqrt(mean_squared_error(predicted_pickups, y_pickup_test))
rmse_back = np.power(10, rmse)
print("RMSE (log-space) = %0.3f" %rmse)
print("RMSE = %0.3f" %rmse_back)

RMSE (log-space) = 0.141
RMSE = 1.382


Our calculated RMSE for our model was 0.141, but that is in log space so we must transform back. Doing so, we obtain a value of 1.382. That means our That means that the other 56% of our predictions are only a factor of 1.382 or less away from the true value. Now remember we are working with a dataset containing several million pickups, so our model being only a factor of 1.382 pickups away from the true pickup values means that model is predicting pickup values very close to the actual number of pickups. Knowing this, now our 43% prediction accuracy is actually quite considerably good.

#### Most important features

In [11]:
import operator
varImp = dict(zip(list(pickup_train.columns.values), reg.feature_importances_))
varImp_sorted = sorted(varImp.items(), key=operator.itemgetter(1), reverse=True)
varImp_sorted

[('Latitude', 0.27621509508875908),
 ('Longitude', 0.23507218037847466),
 ('Year_num', 0.050127152421569302),
 ('Year_cos', 0.050009352118430349),
 ('Year_sin', 0.049514385038255677),
 ('Day_num', 0.049068821142499879),
 ('Day_cos', 0.0310088031001606),
 ('Month_num', 0.029413920719945082),
 ('Day', 0.029375558051922376),
 ('Month_cos', 0.028045684698258282),
 ('Hour_num', 0.02783126044194419),
 ('Day_sin', 0.027064419463383539),
 ('Month_sin', 0.026630089073298717),
 ('Hour', 0.025994081524676652),
 ('Hour_sin', 0.025958827259884246),
 ('Month', 0.019470573634985684),
 ('Hour_cos', 0.019199795843551602)]

#### Write predicted values to csv file
We will take our predicted pickups and output these values to a csv to create visualizations. We will be using Tableau to visualize our predicted values.

In [12]:
predicted_df = pd.DataFrame({'Latitude': pickup_test['Latitude'],
                             'Longitude': pickup_test['Longitude'],
                             'Year': np.repeat(2017, pickup_test.shape[0]),
                             'Day': pickup_test['Day'],
                             'Hour': pickup_test['Hour'],
                             'Month': pickup_test['Month'],
                             'WDay': x_test['WDay'],
                             'YDay': np.round(pickup_test['Year_num']*365),
                             'Pickups': np.round(np.power(10, predicted_pickups)-1, decimals=0)})
predicted_df.to_csv('PredictedPickups.csv', index=False)